In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url1 = 'https://en.wikipedia.org/wiki/202'
url2 = '%E2%80%932'
url3 = '_La_Liga'

dataframes = []

for i in range(4):
    if(i == 2):
        break
    url = url1 + str(i) + url2 + str(i + 1) + url3
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    league_table_header = soup.find('h2', id='League_table')
    table = league_table_header.find_next('table')

    headers = ['Name', 'Test', 'TN']
    count = 0
    for th in table.find_all('th'):
        if(count < 10):
            headers.append(th.text.strip())
            count += 1

    data = []
    for row in table.find_all('tr'):
        cols = row.find_all(['td', 'a'])
        cols = [col.text.strip() for col in cols]
        data.append(cols)

    df = pd.DataFrame(data, columns=headers)
    df = df.rename(columns={'Teamvte': 'Team'})
    df = df.shift(axis=1, periods=3)
    df = df.drop(['Name', 'Test', 'TN'], axis=1)
    df = df.dropna()

    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

numeric_columns = ['Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']
combined_df[numeric_columns] = combined_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

aggregated_df = combined_df.groupby('Team').agg({
    'Pld': 'sum',
    'W': 'sum',
    'D': 'sum',
    'L': 'sum',
    'GF': 'sum',
    'GA': 'sum'
}).reset_index()

print(aggregated_df)

aggregated_df.to_csv('la_liga_statistics.csv', index=False)

               Team  Pld   W   D   L   GF   GA
0            Alavés   76  17  18  41   67  122
1   Athletic Bilbao   76  25  26  25   89   78
2   Atlético Madrid   76  47  16  13  132   68
3         Barcelona   76  45  17  14  153   76
4        Celta Vigo   76  26  21  29   98  100
5             Cádiz   76  19  26  31   71  109
6             Eibar   38   6  12  20   29   52
7             Elche   76  19  21  36   74  107
8          Espanyol   38  10  12  16   40   53
9            Getafe   76  17  26  33   61   84
10          Granada   76  21  21  34   91  126
11           Huesca   38   7  13  18   34   53
12          Levante   76  17  25  34   97  133
13         Mallorca   38  10   9  19   36   63
14          Osasuna   76  23  22  31   74   99
15   Rayo Vallecano   38  11   9  18   39   50
16       Real Betis   76  36  18  22  112   90
17      Real Madrid   76  51  17   8  147   59
18    Real Sociedad   76  34  22  20   99   75
19          Sevilla   76  42  21  13  106   63
20         Va

In [17]:
import pandas as pd
import numpy as np
import csv

# Load data from CSV
datafile = open('la_liga_statistics.csv', 'r')
datareader = csv.reader(datafile)
data = []
for row in datareader:
    data.append(row)

df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df[1:]

